In [1]:
!pwd

/home/kionkim/work/stat-story/Soft_Decision_Tree


In [2]:
import os
import time

import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [48]:
args = {}
args['batch_size'] = 64
args['epochs'] = 40
args['input_dim'] = 784
args['lmbda'] = 0.1
args['log_interval'] = 10
args['lr'] = 0.01
args['max_depth'] = 8
args['momentum'] = 0.5
args['no_cuda'] = False
args['cuda'] = True
args['output_dim'] = 10
args['seed'] = 1

In [101]:
class InnerNode():

    def __init__(self, depth, **kwargs):
        self.args = kwargs
        
        self.fc = nn.Linear(self.args['input_dim'], 1)
        beta = torch.randn(1)
        #beta = beta.expand((self.args.batch_size, 1))
        if self.args['cuda']:
            beta = beta.cuda()
        self.beta = nn.Parameter(beta)
        self.leaf = False
        self.prob = None
        self.leaf_accumulator = []
        self.lmbda = self.args['lmbda'] * 2 ** (-depth)
        self.build_child(depth)
        self.penalties = []

    def reset(self):
        self.leaf_accumulator = []
        self.penalties = []
        self.left.reset()
        self.right.reset()

    def build_child(self, depth):
        if depth < self.args['max_depth']:
            self.left = InnerNode(depth+1, **self.args)
            self.right = InnerNode(depth+1, **self.args)
        else :
            self.left = LeafNode(self.args)
            self.right = LeafNode(self.args)

    def forward(self, x):
        return(F.sigmoid(self.beta*self.fc(x)))
    
    def select_next(self, x):
        prob = self.forward(x)
        if prob < 0.5:
            return(self.left, prob)
        else:
            return(self.right, prob)

    def cal_prob(self, x, path_prob):
        self.data = x
        self.prob = self.forward(x) #probability of selecting right node
        self.path_prob = path_prob
        left_leaf_accumulator = self.left.cal_prob(x, path_prob * (1-self.prob))
        right_leaf_accumulator = self.right.cal_prob(x, path_prob * self.prob)
        self.leaf_accumulator.extend(left_leaf_accumulator)
        self.leaf_accumulator.extend(right_leaf_accumulator)
        return(self.leaf_accumulator)

    def get_penalty(self):
        penalty = (torch.sum(self.prob * self.path_prob) / torch.sum(self.path_prob), self.lmbda)
        if not self.left.leaf:
            left_penalty = self.left.get_penalty()
            right_penalty = self.right.get_penalty()
            self.penalties.append(penalty)
            self.penalties.extend(left_penalty)
            self.penalties.extend(right_penalty)
        return(self.penalties)

In [102]:
class LeafNode():
    def __init__(self, args):
        self.args = args
        self.param = torch.randn(self.args['output_dim'])
        if self.args['cuda']:
            self.param = self.param.cuda()
        self.param = nn.Parameter(self.param)
        self.leaf = True
        self.softmax = nn.Softmax()

    def forward(self):
        return(self.softmax(self.param.view(1,-1)))

    def reset(self):
        pass

    def cal_prob(self, x, path_prob):
        self.data = x
        Q = self.forward()
        #Q = Q.expand((self.args.batch_size, self.args.output_dim))
        Q = Q.expand((path_prob.size()[0], self.args['output_dim']))
        return([[path_prob, Q]])

In [125]:
import torch
import torch.utils.data
from torchvision import datasets, transforms

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)


In [126]:
torch.manual_seed(args['seed'])
if args['cuda']:
    torch.cuda.manual_seed(args['seed'])

try:
    os.makedirs('./data')
except:
    print('directory ./data already exists')

kwargs = {'num_workers': 1, 'pin_memory': True} if args['cuda'] else {}

directory ./data already exists


In [127]:
model = SoftDecisionTree(**args)

In [128]:
#        self.fc = nn.Linear(self.args['input_dim'], 1)
#        beta = torch.randn(1)
        #beta = beta.expand((self.args.batch_size, 1))
#        self.beta = nn.Parameter(beta)
#        self.leaf = False
#        self.prob = None
#        self.leaf_accumulator = []
#        self.lmbda = self.args['lmbda'] * 2 ** (-depth)
#        self.build_child(depth)
#        self.penalties = []

In [129]:
import numpy as np
model.root.left.path_prob

AttributeError: 'InnerNode' object has no attribute 'path_prob'

In [130]:
if args['cuda']:
    model.cuda()

for epoch in range(1, args['epochs'] + 1):
    model.train_(train_loader, epoch)
    model.test_(test_loader, epoch)
save_result()

/home/kionkim/anaconda3/envs/kion_env_pytorch/lib/python3.6/site-packages/torch/autograd/__init__.py:92: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.976123, Accuracy: 5/64 (7.8125%)
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.937533, Accuracy: 19/64 (29.6875%)
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.893087, Accuracy: 12/64 (18.7500%)
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.868050, Accuracy: 17/64 (26.5625%)


Process Process-5:
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/kionkim/anaconda3/envs/kion_env_pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/kionkim/anaconda3/envs/kion_env_pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kionkim/anaconda3/envs/kion_env_pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/home/kionkim/anaconda3/envs/kion_env_pytorch/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/kionkim/anaconda3/envs/kion_env_pytorch/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/kionkim/anaconda3/envs/kion_env_pytorch/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File 

KeyboardInterrupt: 